In [1]:
import numpy as np
import pandas as pd

In [2]:
import keras
print(keras.__version__)

2024-07-24 20:59:09.707507: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 20:59:09.734520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 20:59:09.734546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 20:59:09.735431: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 20:59:09.739998: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 20:59:09.740367: I tensorflow/core/platform/cpu_feature_guard.cc:1

3.4.1


In [4]:
import keras
print(keras.__version__)

3.4.1


In [5]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [6]:
train = pd.read_csv('./data/petfinder_train.csv')
test =  pd.read_csv('./data/petfinder_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/petfinder_train.csv'

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
dict = {
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 0
}

train['Target']= train['AdoptionSpeed'].replace(dict)

In [ ]:
train.drop(columns = ['AdoptionSpeed','Description'], inplace = True)
test.drop(columns = ['Description'], inplace = True)

In [ ]:
train.info()

In [ ]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['MaturitySize'] = le.fit_transform(train['MaturitySize'])
test['MaturitySize'] = le.transform(test['MaturitySize'])

train['FurLength'] = le.fit_transform(train['FurLength'])
test['FurLength'] = le.transform(test['FurLength'])

train['Health'] = le.fit_transform(train['Health'])
test['Health'] = le.transform(test['Health'])

In [ ]:
train['Type'] = le.fit_transform(train['Type'])
test['Type'] = le.transform(test['Type'])

train['Gender'] = le.fit_transform(train['Gender'])
test['Gender'] = le.transform(test['Gender'])

In [ ]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = binary_encoder.fit_transform(train['Breed1'])
test_binary_Breed1 = binary_encoder.transform(test['Breed1'])

train_binary_Color1 = binary_encoder.fit_transform(train['Color1'])
test_binary_Color1 = binary_encoder.transform(test['Color1'])

train_binary_Color2 = binary_encoder.fit_transform(train['Color2'])
test_binary_Color2 = binary_encoder.transform(test['Color2'])

train_binary_Vaccinated = binary_encoder.fit_transform(train['Vaccinated'])
test_binary_Vaccinated = binary_encoder.transform(test['Vaccinated'])

train_binary_Sterilized = binary_encoder.fit_transform(train['Sterilized'])
test_binary_Sterilized = binary_encoder.transform(test['Sterilized'])

In [ ]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

test = pd.concat([test, test_binary_Breed1, test_binary_Color1, test_binary_Color2, test_binary_Vaccinated, test_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
test.drop(columns = columns, inplace = True)

train.info()

In [ ]:
for column in train.drop(columns = ['Target']).columns:
  mean= train[column].mean() #To-Do (Mean of train_data[column])
  std=train[column].std() #To-Do (Standard Deviation of train_data[column])
  train[column]=(train[column]- mean)/std
  test[column]=(test[column] - mean)/std

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns = ['Target']), train['Target'], test_size = 0.1)

print('Train examples: ', len(X_train), len(y_train))
print('Validation examples: ', len(X_valid), len(y_valid))
print('Test examples: ',  len(test))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(26,)))
model.add(keras.layers.Dense(5000, activation='relu'))
model.add(keras.layers.Dense(1000, activation='relu'))
model.add(keras.layers.Dense(500, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy'])

In [ ]:
epochs = 10
BATCH_SIZE = 128

history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    validation_data=(X_valid, y_valid))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid, model.predict(X_valid) > 0.5, average='weighted')

In [ ]:
submission = pd.DataFrame(model.predict(test) > 0.5, columns = ['Target'])
submission